In [1]:
from arcgis.gis import GIS
from IPython.display import display
import random
gis = GIS("http://geodata.maps.arcgis.com", "student_morten", "SomStud2018")

In [2]:
# searches for Dovrebanen feature layer where the train will run
search_result = gis.content.search('title:Dovrebanen_Points', item_type = "Feature Layer")
dovre_layer = search_result[0].layers
dovre_flayer = dovre_layer[0]
dovre_features = dovre_layer[0].query()

In [3]:
# search for the train object and the layer to add warnings to
search_result = gis.content.search('title:Moving_Trains', item_type = "Feature Layer")
train_layer = search_result[0].layers
train_flayer = train_layer[0]

search_result = gis.content.search('title:Dovrebanen_Warning', item_type = "Feature Layer")
warning_layer = search_result[0].layers
warning_flayer = warning_layer[0]

In [4]:
def remove_all_warnings():
    # Makes sure no warning is shown at the start of the script
    for warning in warning_layer[0].query().features:
        warning_flayer.edit_features(deletes=str(warning.attributes["OBJECTID"]))

    warning_index1 = 360
    warning_index2 = 600

    warning1 = dovre_features.features[warning_index1]
    warning1.attributes["OBJECTID"] = 1

    warning2 = dovre_features.features[warning_index2]
    warning2.attributes["OBJECTID"] = 2

In [ ]:
def moving_train():
    # scripts starts before Tangen station
    start = 200
    end = 440
    index = start
    
    warning_index1 = 313
    warning_index2 = 330

    warning1 = dovre_features.features[warning_index1]
    warning1.attributes["OBJECTID"] = 1

    warning2 = dovre_features.features[warning_index2]
    warning2.attributes["OBJECTID"] = 2
    
    while True:
        if index == 280:
            warning_flayer.edit_features(adds=[warning1])
        if index == warning_index2-20:
            warning_flayer.edit_features(adds=[warning2])
        
        tfo_feature = dovre_features.features[index]
        tfo_feature.attributes["OBJECTID"] = 1
        update_result = train_flayer.edit_features(updates=[tfo_feature])                    
        
        # Controls how fast the train goes each 6 seconds
        if index < 260 or index > 360:
            index += 40
        elif index <= 280 or index > warning_index2+10:
            index += 20
        else:
            index += 5
        
        time.sleep(6)

        # hacks to remove the first warning before the seconds one
        if index >= warning_index1+10 or index >= warning_index2+10:
            for warning in warning_layer[0].query().features:
                warning_flayer.edit_features(deletes=str(warning.attributes["OBJECTID"]))
                break
       
    # Resets script
        if index > end:
            index = start
            for warning in warning_layer[0].query().features:
                warning_flayer.edit_features(deletes=str(warning.attributes["OBJECTID"]))

In [ ]:
# Starts the scripts
remove_all_warnings()
moving_train()